# Phase 1 — Software Engineer Salaries (Data Import, Cleaning, Transformation)

This notebook reproduces the steps used to import, clean, and transform the uploaded raw dataset `Software Engineer Salaries.csv`.

Files produced:
- `Cleaned_Software_Engineer_Salaries.csv`
- `Phase1_documentation.md`


In [ ]:
import pandas as pd
df = pd.read_csv(r"/mnt/data/Software Engineer Salaries.csv")
print('Initial shape:', df.shape)
df.head()

In [ ]:
# Basic inspection
print('Columns:', list(df.columns))
print(df.info())
print(df.isna().sum().sort_values(ascending=False).head(20))

## Cleaning steps applied

1. Drop fully empty columns.
2. Drop duplicate rows.
3. Parse salary-like columns into numeric `salary_parsed_raw` (handle ranges and 'K' suffix).
4. Standardize job title and location text fields.
5. Create numeric years of experience where available and bucket into bands.
6. Flag extreme outliers.


In [ ]:
import re
import numpy as np

def parse_salary_to_number(s):
    if pd.isna(s):
        return np.nan
    if isinstance(s, (int, float, np.integer, np.floating)):
        return float(s)
    s = str(s).replace(',', '').strip()
    nums = re.findall(r"\d+(?:\.\d+)?", s)
    if not nums:
        return np.nan
    if re.search(r'k\b', s, re.I):
        vals = []
        for m in re.finditer(r"(\d+(?:\.\d+)?)\s*[kK]?", s):
            num = float(m.group(1))
            span = s[m.start():m.end()]
            if re.search(r'[kK]', span):
                num = num * 1000
            vals.append(num)
        if vals:
            return float(np.mean(vals))
    nums_f = [float(x) for x in nums]
    if len(nums_f) == 1:
        return nums_f[0]
    else:
        return float(np.mean(nums_f))

# Example: detect a candidate salary column automatically (first match)
salary_candidates = [c for c in df.columns if re.search(r"salary|comp|pay|wage", c, re.I)]
salary_col = salary_candidates[0] if salary_candidates else None
print('Detected salary column:', salary_col)

df_clean = df.copy()
df_clean = df_clean.dropna(axis=1, how='all')
df_clean = df_clean.drop_duplicates()
if salary_col:
    df_clean['salary_parsed_raw'] = df_clean[salary_col].apply(parse_salary_to_number)
else:
    df_clean['salary_parsed_raw'] = pd.NA

# Standardize text fields
for c in df_clean.columns:
    if re.search(r"job|title|position|role", c, re.I):
        df_clean['job_title_clean'] = df_clean[c].astype(str).str.strip().str.title()
        break
for c in df_clean.columns:
    if re.search(r"location|country|city|employee_residence|company_location", c, re.I):
        df_clean['location_clean'] = df_clean[c].astype(str).str.strip().str.title()
        break

# Save cleaned CSV
df_clean.to_csv(r"{str(CLEANED_CSV)}", index=False)
print('Cleaned saved to:', r"{str(CLEANED_CSV)}")

In [ ]:
# Example transformations
# Average salary by location (if salary_parsed_raw exists)
if 'salary_parsed_raw' in df_clean.columns:
    avg_by_loc = df_clean.groupby('location_clean', dropna=True)['salary_parsed_raw'].mean().reset_index().sort_values('salary_parsed_raw', ascending=False)
    print(avg_by_loc.head(20))
else:
    print('No parsed salary column available')